### Heroes Of Pymoli Data Analysis
* Of the 1163 active players, the vast majority are male (84%). There also exists, a smaller, but notable proportion of female players (14%).

* Our peak age demographic falls between 20-24 (44.8%) with secondary groups falling between 15-19 (18.60%) and 25-29 (13.4%).  
-----

### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import pandas as pd
import numpy as np

# File to Load (Remember to Change These)
file_to_load = "Resources/purchase_data.csv"

# Read Purchasing File and store into Pandas data frame
purchase_data = pd.read_csv(file_to_load)

## Player Count

* Display the total number of players


In [2]:
total = purchase_data.SN.nunique() 
Player_Count = pd.DataFrame([{"Total Player": total}])
Player_Count

,Total Player
0,576


## Purchasing Analysis (Total)

* Run basic calculations to obtain number of unique items, average price, etc.


* Create a summary data frame to hold the results


* Optional: give the displayed data cleaner formatting


* Display the summary data frame


In [3]:
items = purchase_data["Item ID"].nunique()
Purchase = purchase_data['Purchase ID'].count()
Total_Revenue = purchase_data['Price'].sum()
Price = Total_Revenue/Purchase

Table = pd.DataFrame([{"Number of Unique Items": items ,"Number of Purchases": Purchase,"Total Revenue":Total_Revenue, "Average Price" : Price}])
Table2 = Table[["Number of Unique Items","Average Price","Number of Purchases","Total Revenue"]]
Table2["Average Price"]= Table2["Average Price"].map("${:,.2f}".format)
Table2['Total Revenue'] = Table2['Total Revenue'].map("${:,.2f}".format)
Table2

,Number of Unique Items,Average Price,Number of Purchases,Total Revenue
0,183,$3.05,780,"$2,379.77"


## Gender Demographics

* Percentage and Count of Male Players


* Percentage and Count of Female Players


* Percentage and Count of Other / Non-Disclosed




In [4]:
#We sort the table and then delete all the duplicates in the table
copy_Table1 = purchase_data.sort_values('SN',ascending =True)
copy_Table2 = copy_Table1.drop_duplicates(subset ="SN",keep = 'first')

#Sum of the total number of times Male, Female, and Other show up in the column "Gender"
Total = copy_Table2['Gender'].count()
Males = (copy_Table2['Gender']=='Male').sum()
Females = (copy_Table2['Gender']=='Female').sum()
Other_Non_Disclosed = (copy_Table2['Gender']=='Other / Non-Disclosed').sum()

#Taking the percentages for each Gender
MP = Males/Total*100
FP = Females/Total*100
OP = Other_Non_Disclosed/Total*100

#We print the dataframe with the values 
Genders = pd.Series(['Male','Female','Other / Non-Disclosed'])
Gender_Demographics = pd.DataFrame({"Total Count":[Males,Females,Other_Non_Disclosed],"Percentage of Players":[MP,FP,OP]})
Gender_Demographics["Percentage of Players"] = Gender_Demographics['Percentage of Players'].map("{:,.2f}".format)
Gender_Demographics = Gender_Demographics.set_index([Genders])

#Display DataFrame
Gender_Demographics

,Total Count,Percentage of Players
Male,484,84.03
Female,81,14.06
Other / Non-Disclosed,11,1.91



## Purchasing Analysis (Gender)

* Run basic calculations to obtain purchase count, avg. purchase price, avg. purchase total per person etc. by gender




* Create a summary data frame to hold the results


* Optional: give the displayed data cleaner formatting


* Display the summary data frame

In [5]:
#We group two times and setting them to seperate variables
Gender_Table1 = purchase_data.groupby("Gender").count()
Gender_Table2 = purchase_data.groupby("Gender").agg({"Price":sum })

#Merge the two variables
Gender_Table_Merge = pd.merge(Gender_Table1,Gender_Table2,on = "Gender")

#Calculate Average Purchase Price
Gender_Table_Merge["SN"] = (Gender_Table_Merge["Price_y"]/Gender_Table_Merge["Purchase ID"])

#Drop unnecessary columns in the DataFrame 
Gender_Table_Merge = Gender_Table_Merge.drop(columns = ["Age","Item ID","Item Name"])

#Rename Columns
Gender_Table_Merge = Gender_Table_Merge.rename(columns = {"Purchase ID" : "Purchase Count","SN": "Average Purchase Price","Price_y":"Total Purchase Value","Price_x": "Avg Total Purchase per Person"})

#ATPP = Gender_Table_Merge["Total Purchase Value"]/Gender_Demographics["Total Count"]
Gender_Table_Merge["Avg Total Purchase per Person"] = (Gender_Table_Merge["Total Purchase Value"]/Gender_Demographics["Total Count"])

#Reordering the column in the DataFrame 
Gender_Table_Merge = Gender_Table_Merge[["Purchase Count","Average Purchase Price","Total Purchase Value","Avg Total Purchase per Person"]]

# #Clean up data with correct decimal places and currency sign
Gender_Table_Merge["Average Purchase Price"] = Gender_Table_Merge["Average Purchase Price"].map("${:,.2f}".format)
Gender_Table_Merge["Total Purchase Value"] = Gender_Table_Merge["Total Purchase Value"].map("${:,.2f}".format)
Gender_Table_Merge["Avg Total Purchase per Person"] = Gender_Table_Merge["Avg Total Purchase per Person"].map("${:,.2f}".format)

#Display DataFrame
Gender_Table_Merge


,Purchase Count,Average Purchase Price,Total Purchase Value,Avg Total Purchase per Person
Gender,,,,
Female,113,$3.20,$361.94,$4.47
Male,652,$3.02,"$1,967.64",$4.07
Other / Non-Disclosed,15,$3.35,$50.19,$4.56


## Age Demographics

* Establish bins for ages


* Categorize the existing players using the age bins. Hint: use pd.cut()


* Calculate the numbers and percentages by age group


* Create a summary data frame to hold the results


* Optional: round the percentage column to two decimal points


* Display Age Demographics Table


In [6]:
player_demos = purchase_data.loc[:,["Gender","SN","Age"]]
player_demos = player_demos.drop_duplicates()

# Create the bins in which Data will be held  
bins = [0, 10, 15, 20, 25, 30, 35, 40, 1000]

# Create the names for the four bins
group_names = ["<10", "10-14", "15-19", "20-24", "25-29","30-34", "35-39","40+"]
player_demos["Age Group"] = pd.cut(player_demos["Age"], bins, labels=group_names,right=False)
AgeTable = player_demos.groupby("Age Group").count()
AgeTable = AgeTable.rename(columns = {"Gender":"Total Count"})

#We take the total number of counts and use Total_Age to divide the value for each Age Group
Total_Age = AgeTable["Total Count"].sum()
AgeTable["SN"] = (AgeTable["Total Count"]/Total_Age*100).map("{:,.2f}".format)

#Drop and rename columns 
AgeTable = AgeTable.drop(columns = "Age")
AgeTable = AgeTable.rename(columns = {"SN": "Percentage of Players"})

#Display new table 
AgeTable

,Total Count,Percentage of Players
Age Group,,
<10,17,2.95
10-14,22,3.82
15-19,107,18.58
20-24,258,44.79
25-29,77,13.37
30-34,52,9.03
35-39,31,5.38
40+,12,2.08


## Purchasing Analysis (Age)

* Bin the purchase_data data frame by age


* Run basic calculations to obtain purchase count, avg. purchase price, avg. purchase total per person etc. in the table below


* Create a summary data frame to hold the results


* Optional: give the displayed data cleaner formatting


* Display the summary data frame

In [7]:
player_demos = purchase_data.loc[:,["Gender","SN","Age","Price"]]
# Create the bins in which Data will be held  
bins = [0,10, 15, 20, 25, 30, 35, 40, 1000]
# Create the names for the four bins
group_names = ["<10", "10-14", "15-19", "20-24", "25-29","30-34", "35-39","40+"]
player_demos["Age Group"] = pd.cut(player_demos["Age"], bins, labels=group_names,right=False)
NewTable = player_demos.groupby("Age Group").count()
Age_Table = player_demos.groupby("Age Group").agg({"Price":sum })

#Merge tables together
Age_Merge = pd.merge(NewTable,Age_Table, on = "Age Group")

#Rename columns in new table
Age_Merge = Age_Merge.rename(columns = {"Gender":"Purchase Count","Price_y":"Total Purchase Value","SN":"Average Purchase Price"})
Age_Merge["Average Purchase Price"] = Age_Merge["Total Purchase Value"]/Age_Merge["Purchase Count"]

#Drop unecessary columns 
Age_Merge = Age_Merge.drop(columns = ["Price_x"])

#Calculate Avg Total Purchase per Person
Age_Merge["Age"] = Age_Merge["Total Purchase Value"]/AgeTable["Total Count"]

#Renaming column
Age_Merge = Age_Merge.rename(columns = {"Age": "Avg Total Purchase per Person"})

#Reordering columns in the DataFrame
Age_Merge = Age_Merge[["Purchase Count","Average Purchase Price","Total Purchase Value","Avg Total Purchase per Person"]]

#Clean dataframe with correct decimal place and currency sign
Age_Merge["Average Purchase Price"] = Age_Merge["Average Purchase Price"].map("${:,.2f}".format)
Age_Merge["Total Purchase Value"] = Age_Merge["Total Purchase Value"].map("${:,.2f}".format)
Age_Merge["Avg Total Purchase per Person"] = Age_Merge["Avg Total Purchase per Person"].map("${:,.2f}".format)

#Move rows around 
Age_Merge = Age_Merge.loc[["10-14","15-19","20-24","25-29","30-34","35-39","40+","<10"]]

#Display DataFrame
Age_Merge


,Purchase Count,Average Purchase Price,Total Purchase Value,Avg Total Purchase per Person
Age Group,,,,
10-14,28,$2.96,$82.78,$3.76
15-19,136,$3.04,$412.89,$3.86
20-24,365,$3.05,"$1,114.06",$4.32
25-29,101,$2.90,$293.00,$3.81
30-34,73,$2.93,$214.00,$4.12
35-39,41,$3.60,$147.67,$4.76
40+,13,$2.94,$38.24,$3.19
<10,23,$3.35,$77.13,$4.54


## Top Spenders

* Run basic calculations to obtain the results in the table below


* Create a summary data frame to hold the results


* Sort the total purchase value column in descending order


* Optional: give the displayed data cleaner formatting


* Display a preview of the summary data frame



In [8]:
#Change the column name from Purchase ID to Purchase Count 
purchase_data2 = purchase_data.rename(columns = {"Purchase ID":"Purchase Count"})

#Group purchase_data2 by 'SN' and count the values of the new index
group1 = purchase_data2.groupby("SN").count()

#Take the sum of each SN value and sum of the total purchase value 
group2 = purchase_data2.groupby("SN").agg({"Price":sum })

#Merge the two DataFrames on "SN"
merge1 = pd.merge(group1,group2,on = "SN")

#Sort the new DataFrame by Price_y
merge1 = merge1.sort_values("Price_y", ascending = False)

#Rename the column "Price_y" to Total Purchase Value 
merge1 = merge1.rename(columns = {'Price_y':"Total Purchase Value"})

#Take Total Purchase Value and divide it by Purchase count to get the Average Purchase Price and append that series in the DataFrame
merge1["Price_x"] = (merge1["Total Purchase Value"]/merge1["Purchase Count"]).map("${:,.2f}".format)
merge1 = merge1.drop(columns = ["Age","Gender","Item ID","Item Name"])
merge1 = merge1.rename(columns = {"Price_x" : "Average Purchase Price"})
merge1["Total Purchase Value"] = merge1["Total Purchase Value"].map("${:,.2f}".format)

#Display DataFrame
merge1.head()


,Purchase Count,Average Purchase Price,Total Purchase Value
SN,,,
Lisosia93,5,$3.79,$18.96
Idastidru52,4,$3.86,$15.45
Chamjask73,3,$4.61,$13.83
Iral74,4,$3.40,$13.62
Iskadarya95,3,$4.37,$13.10


## Most Popular Items

* Retrieve the Item ID, Item Name, and Item Price columns


* Group by Item ID and Item Name. Perform calculations to obtain purchase count, item price, and total purchase value


* Create a summary data frame to hold the results


* Sort the purchase count column in descending order


* Optional: give the displayed data cleaner formatting


* Display a preview of the summary data frame



In [9]:
purchase_data3 = purchase_data.rename(columns = {"Purchase ID": "Purchase Count"})
#Create two variable that will hold two tables with the same groupby
grouping1 = purchase_data3.groupby(["Item ID","Item Name"]).count()
grouping2 = purchase_data3.groupby(["Item ID","Item Name"]).agg({"Price":sum})

#Sort the values in the first groupby table
grouping1 = grouping1.sort_values("Purchase Count",ascending = False)

#Merge the two tables
mergeing1 = pd.merge(grouping1,grouping2, on = ["Item ID","Item Name"])

#Rename the columns in the merge 
mergeing1 = mergeing1.rename(columns = {"Price_y": "Total Purchase Value"})

#Clean up and format the merge 
mergeing1['Price_x'] = (mergeing1["Total Purchase Value"]/mergeing1["Purchase Count"]).map("${:,.2f}".format)

#Drop unnecessary columns
mergeing1 = mergeing1.drop(columns = ["SN","Age","Gender"])

#Rename columns 
mergeing1 = mergeing1.rename(columns = {"Price_x": "Item Price"})

#Clean up and format the merge 
mergeing1["Total Purchase Value"] = mergeing1["Total Purchase Value"].map("${:,.2f}".format)

#Print the final table
mergeing1.head()


,,Purchase Count,Item Price,Total Purchase Value
Item ID,Item Name,,,
178,"Oathbreaker, Last Hope of the Breaking Storm",12,$4.23,$50.76
145,Fiery Glass Crusader,9,$4.58,$41.22
108,"Extraction, Quickblade Of Trembling Hands",9,$3.53,$31.77
82,Nirvana,9,$4.90,$44.10
19,"Pursuit, Cudgel of Necromancy",8,$1.02,$8.16


## Most Profitable Items

* Sort the above table by total purchase value in descending order


* Optional: give the displayed data cleaner formatting


* Display a preview of the data frame



In [11]:
#----------------------Same steps as before-------------------------------------------------------------------
purchase_data3 = purchase_data.rename(columns = {"Purchase ID": "Purchase Count"})
grouping1 = purchase_data3.groupby(["Item ID","Item Name"]).count()
grouping2 = purchase_data3.groupby(["Item ID","Item Name"]).agg({"Price":sum})
grouping1 = grouping1.sort_values("Purchase Count",ascending = False)
mergeing1 = pd.merge(grouping1,grouping2, on = ["Item ID","Item Name"])
mergeing1 = mergeing1.rename(columns = {"Price_y": "Total Purchase Value"})
mergeing1['Price_x'] = (mergeing1["Total Purchase Value"]/mergeing1["Purchase Count"]).map("${:,.2f}".format)
mergeing1 = mergeing1.drop(columns = ["SN","Age","Gender"])
mergeing1 = mergeing1.rename(columns = {"Price_x": "Item Price"})
#---------------------------------------------------------------------------------------------------------------
#Sort the table by Total Purchase Value
mergeing1 = mergeing1.sort_values("Total Purchase Value",ascending = False)

#Format the table
mergeing1["Total Purchase Value"] = mergeing1["Total Purchase Value"].map("${:,.2f}".format)

#Print the table 
mergeing1.head()

,,Purchase Count,Item Price,Total Purchase Value
Item ID,Item Name,,,
178,"Oathbreaker, Last Hope of the Breaking Storm",12,$4.23,$50.76
82,Nirvana,9,$4.90,$44.10
145,Fiery Glass Crusader,9,$4.58,$41.22
92,Final Critic,8,$4.88,$39.04
103,Singed Scalpel,8,$4.35,$34.80


## written description of three observable trends based on the data.

* The data shows that there are a greater number of males that play this game and have a higher total purchase value than other genders.

* The data demonstrates that the Total Purchase Value for some of the categories does not exceed $51.00

* The data also shows the average price for whichever category or grouping does not exceed $4.00. 


